In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from new_distance import *
%load_ext autoreload
%autoreload 2

Considering the possible effects connected to LC50. We need to filter on **Mortality Group**

## Preprocessing on base_db

In [19]:
base_db = pd.read_csv("use_db_sabato14.csv")
base_db.head()

,test_cas,exposure_type,obs_duration_mean,conc1_type,conc1_mean,class,tax_order,family,genus,species,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,10108-64-2,S,48.0,T,46.900,Actinopterygii,Cypriniformes,Cyprinidae,Carassius,auratus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
1,10108-64-2,S,24.0,T,200.000,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Fundulus,heteroclitus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
2,10108-64-2,F,96.0,T,0.001,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
3,10108-64-2,S,96.0,T,15.900,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Cyprinodon,variegatus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
4,10108-64-2,F,694.0,T,0.030,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945


In [20]:
base_db = base_db[base_db.obs_duration_mean.isin([24, 48, 72, 96])]

In [21]:
to_drop = base_db[base_db.bonds_number.isnull()].index

In [22]:
base_db = base_db.drop(to_drop,axis=0)

In [23]:
base_db.columns

Index(['test_cas', 'exposure_type', 'obs_duration_mean', 'conc1_type',
       'conc1_mean', 'class', 'tax_order', 'family', 'genus', 'species',
       'atom_number', 'alone_atom_number', 'tripleBond', 'doubleBond',
       'bonds_number', 'ring_numer', 'Mol', 'MorganDensity', 'LogP'],
      dtype='object')

In [24]:
use_db = base_db.copy()
use_db = use_db[['test_cas', 'exposure_type', 'obs_duration_mean', 'conc1_type',
       'conc1_mean', 'class', 'tax_order', 'family', 'genus', 'species','atom_number', 'alone_atom_number', 'tripleBond',
       'doubleBond', 'bonds_number', 'ring_numer',
       'Mol', 'MorganDensity', 'LogP']]

In [25]:
db_species = use_db[["species", 'class', 'tax_order', 'family', 'genus']]

In [26]:
use_db = use_db.groupby(by=["test_cas", "species", "conc1_type", "exposure_type", "obs_duration_mean"]).agg("median").reset_index()
use_db.index.size

23332

In [27]:
db_species = db_species.groupby("species").first()

In [28]:
use_db = use_db.merge(db_species, on="species")

### Simple classification with Smile Features
Considering a threshold of 1mg/L of mortality

In [29]:
X = use_db.copy()
X = X[[
 'exposure_type',
 'obs_duration_mean',
 'conc1_type',
 'species',
    'class', 'tax_order', 'family', 'genus',
 'atom_number', 
    'alone_atom_number', 
    'tripleBond',
       'doubleBond',
 'bonds_number',
  'ring_numer',
  'Mol', 
  'MorganDensity', 'LogP']]

In [30]:
use_db

,test_cas,species,conc1_type,exposure_type,obs_duration_mean,conc1_mean,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP,class,tax_order,family,genus
0,100-00-5,rerio,F,F,48.0,15.00,0.317908,2,0,1,0.488106,1.0,0.082505,1.300000,2.2482,Actinopterygii,Cypriniformes,Cyprinidae,Danio
1,100-00-5,rerio,F,F,96.0,15.00,0.317908,2,0,1,0.488106,1.0,0.082505,1.300000,2.2482,Actinopterygii,Cypriniformes,Cyprinidae,Danio
2,100-01-6,rerio,F,S,96.0,87.60,0.317908,2,0,1,0.488106,1.0,0.072092,1.300000,1.1770,Actinopterygii,Cypriniformes,Cyprinidae,Danio
3,100-02-7,rerio,F,AQUA,48.0,26.50,0.317908,2,0,1,0.488106,1.0,0.072622,1.300000,1.3004,Actinopterygii,Cypriniformes,Cyprinidae,Danio
4,100-44-7,rerio,F,S,96.0,4.00,0.217006,0,0,0,0.447258,1.0,0.065856,1.125000,2.4254,Actinopterygii,Cypriniformes,Cyprinidae,Danio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23327,94-75-7,boga,F,R,96.0,3.80,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,Actinopterygii,Cypriniformes,Cyprinidae,Labeo
23328,94-75-7,neilgherriensis,F,R,24.0,7.80,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,Actinopterygii,Cypriniformes,Cyprinidae,Rasbora
23329,94-75-7,neilgherriensis,F,R,48.0,6.80,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,Actinopterygii,Cypriniformes,Cyprinidae,Rasbora
23330,94-75-7,neilgherriensis,F,R,96.0,5.60,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,Actinopterygii,Cypriniformes,Cyprinidae,Rasbora


In [31]:
X['exposure_type'].unique()

array(['F', 'S', 'AQUA', 'R', 'E', 'SP', 'P', 'O', 'IP', 'FD'],
      dtype=object)

In [32]:
X['conc1_type'].unique()

array(['F', 'D', 'T', 'A', 'U', 'L'], dtype=object)

In [33]:
len(X['species'].unique())

475

For this  vaiables we decided to transform than by an OrdinalEncoder, in order to have all numnerical variables

In [34]:
enc = OrdinalEncoder(dtype=int)
enc.fit(X[["exposure_type", "conc1_type","species", 'class', 'tax_order', 'family', 'genus']])
X[["exposure_type", "conc1_type","species",  'class', 'tax_order', 'family', 'genus']] = enc.transform(X[["exposure_type", "conc1_type","species",  'class', 'tax_order', 'family', 'genus']]) + 1

In [35]:
X.head()

,exposure_type,obs_duration_mean,conc1_type,species,class,tax_order,family,genus,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,3,48.0,3,379,1,10,35,80,0.317908,2,0,1,0.488106,1.0,0.082505,1.300,2.2482
1,3,96.0,3,379,1,10,35,80,0.317908,2,0,1,0.488106,1.0,0.082505,1.300,2.2482
2,9,96.0,3,379,1,10,35,80,0.317908,2,0,1,0.488106,1.0,0.072092,1.300,1.1770
3,1,48.0,3,379,1,10,35,80,0.317908,2,0,1,0.488106,1.0,0.072622,1.300,1.3004
4,9,96.0,3,379,1,10,35,80,0.217006,0,0,0,0.447258,1.0,0.065856,1.125,2.4254


We also devide the label, in order to have a binary classification

In [42]:
y = use_db["conc1_mean"].copy().values
y = np.where(y >= 1, 1, 0)

In [43]:
print(len(y[y==0]), len(y[y==1]))

8424 14908


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## KNN

We first implement a classification by a K Neighbors Classifier

In [45]:
neigh = KNeighborsClassifier()
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)

In [46]:
accuracy_score(y_test, y_pred)

0.734025974025974

## Random Forest

In [49]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train.ravel())
y_pred = forest.predict(X_test)

C:\Users\Manuel\Anaconda3\envs\ml\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [50]:
accuracy_score(y_test, y_pred)

0.9062337662337663

This is the second model we have implemented, with an accurancy of the 0.91. which for the moment is not bad

## Gradient Boosting

In [40]:
grboost = GradientBoostingClassifier()
grboost.fit(X_train, y_train.ravel())
y_pred = grboost.predict(X_test)

In [41]:
accuracy_score(y_test, y_pred)

0.8331623931623932

In [42]:
# grid = GridSearchCV(forest, param_grid={"n_estimators": list(range(1, 500))}, scoring="accuracy", cv=5)
# grid.fit(X_train, y_train.ravel())

## bozza, NON RUNNARE!!!

In [44]:
df = pd.DataFrame(columns = X.columns, index=['KNN','Random Forest','Gradient Boosting'])

In [45]:
for i in X.columns:
    new_train = X_train.copy()
    new_test = X_test.copy()
    del new_train[i]
    del new_test[i]
    l=[]
    neigh = KNeighborsClassifier()
    neigh.fit(new_train, y_train.ravel())
    y_pred = neigh.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    forest = RandomForestClassifier(n_estimators = 100)
    forest.fit(new_train, y_train.ravel())
    y_pred = forest.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    grboost = GradientBoostingClassifier()
    grboost.fit(new_train, y_train.ravel())
    y_pred = grboost.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    df[i] = l

KeyboardInterrupt: 

## How doea the accurancy change without the column-variale?


In [54]:
corr = X.corr()
corr = corr.round(2)
corr.style.background_gradient(cmap='coolwarm')

,exposure_type,obs_duration_mean,conc1_type,species,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
exposure_type,1,-0.06,-0.13,-0,0.16,0.02,-0.04,0.09,0.16,0.09,0.13,0.03,0.07
obs_duration_mean,-0.06,1,0.01,0.02,-0.01,0.01,0.03,0,-0.06,-0.04,-0.02,-0,-0.05
conc1_type,-0.13,0.01,1,-0.09,-0.41,0.35,-0.02,-0.1,-0.68,-0.38,-0.36,-0.06,-0.59
species,-0,0.02,-0.09,1,-0.01,-0.01,-0.01,0,0.03,0.01,-0.01,0.01,0.02
atom_number,0.16,-0.01,-0.41,-0.01,1,0.17,0.03,0.6,0.74,0.43,0.76,-0.02,0.38
alone_atom_number,0.02,0.01,0.35,-0.01,0.17,1,-0.06,0.48,-0.11,-0,0.25,-0.12,-0.48
tripleBond,-0.04,0.03,-0.02,-0.01,0.03,-0.06,1,-0.07,0.04,0.05,0.03,0.09,0.07
doubleBond,0.09,0,-0.1,0,0.6,0.48,-0.07,1,0.42,0.24,0.51,0.13,-0.01
bonds_number,0.16,-0.06,-0.68,0.03,0.74,-0.11,0.04,0.42,1,0.64,0.72,-0.03,0.68
ring_numer,0.09,-0.04,-0.38,0.01,0.43,-0,0.05,0.24,0.64,1,0.69,-0.26,0.49


In [45]:
df

,exposure_type,obs_duration_mean,conc1_type,species,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
KNN,0.855328,0.857037,0.851339,0.859259,0.851966,0.848148,0.850484,0.845641,0.851624,0.850313,0.849174,0.849573,0.829231
Random Forest,0.906610,0.913903,0.910655,0.884217,0.912593,0.913390,0.913105,0.912650,0.911681,0.912536,0.901311,0.912308,0.911396
Gradient Boosting,0.829972,0.830085,0.826154,0.831054,0.831510,0.830085,0.829972,0.827578,0.828604,0.828547,0.812194,0.831966,0.817436


In [68]:
df1 = pd.DataFrame(columns = X.columns, index=['KNN','Random Forest','Gradient Boosting'])

In [59]:
new_train = X_train.copy()
new_test = X_test.copy()
for i in X.columns:
    
    l=[]
    neigh = KNeighborsClassifier()
    neigh.fit(new_train, y_train.ravel())
    y_pred = neigh.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    forest = RandomForestClassifier(n_estimators = 100)
    forest.fit(new_train, y_train.ravel())
    y_pred = forest.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    grboost = GradientBoostingClassifier()
    grboost.fit(new_train, y_train.ravel())
    y_pred = grboost.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    df1[i] = l
    del new_train[i]
    del new_test[i]    
    

In [73]:

for i in range(1,len(X.columns)):
    col = list(X.columns[:i])
    l=[]
    new_train = X_train[col]
    new_test = X_test[col]
    
    neigh = KNeighborsClassifier()
    neigh.fit(new_train, y_train.ravel())
    y_pred = neigh.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    forest = RandomForestClassifier(n_estimators = 100)
    forest.fit(new_train, y_train.ravel())
    y_pred = forest.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    grboost = GradientBoostingClassifier()
    grboost.fit(new_train, y_train.ravel())
    y_pred = grboost.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    df1[X.columns[i]] = l 

In [74]:
df1

,exposure_type,obs_duration_mean,conc1_type,species,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
KNN,NaN,0.440627,0.497208,0.573048,0.657037,0.744444,0.764729,0.771624,0.796467,0.803305,0.820513,0.824957,0.829231
Random Forest,NaN,0.586838,0.603305,0.623476,0.685698,0.772422,0.810085,0.819772,0.853732,0.883875,0.883989,0.910826,0.912137
Gradient Boosting,NaN,0.586838,0.601481,0.616695,0.634587,0.702051,0.708262,0.712934,0.740855,0.773675,0.778860,0.814473,0.817436


## KNN

In [91]:
import itertools
to_compare = list(itertools.combinations(list(X_train.columns), 2))

In [92]:
df2 = pd.DataFrame(columns = ['Accurancy'], index=to_compare)

In [93]:
for i in df2.index:
    
    new_train = X_train[list(i)]
    new_test = X_test[list(i)]
    
    neigh = KNeighborsClassifier()
    neigh.fit(new_train, y_train.ravel())
    y_pred = neigh.predict(new_test)
    df2['Accurancy'].loc[i] = accuracy_score(y_test, y_pred)

In [68]:
df2.sort_values(by= ['Accurancy'],ascending=False).head(20)

,Accurancy
"(species, Mol)",0.849345
"(species, LogP)",0.849003
"(bonds_number, Mol)",0.847749
"(ring_numer, Mol)",0.846097
"(exposure_type, Mol)",0.842108
"(atom_number, Mol)",0.841368
"(tripleBond, Mol)",0.838746
"(conc1_type, Mol)",0.837436
"(doubleBond, Mol)",0.835783
"(Mol, LogP)",0.83208


## RANDOM FOREST

In [69]:
df3 = pd.DataFrame(columns = ['Accurancy_RF'], index=to_compare)

In [72]:
for i in df3.index:
    
    new_train = X_train[list(i)]
    new_test = X_test[list(i)]
    
    forest = RandomForestClassifier(n_estimators = 100)
    forest.fit(new_train, y_train.ravel())
    y_pred = forest.predict(new_test)
    df3['Accurancy_RF'].loc[i] = accuracy_score(y_test, y_pred)

In [74]:
df3.sort_values(by= ['Accurancy_RF'],ascending=False).head(20)

,Accurancy_RF
"(species, Mol)",0.890484
"(species, LogP)",0.882906
"(exposure_type, Mol)",0.869516
"(conc1_type, Mol)",0.862735
"(Mol, LogP)",0.861425
"(ring_numer, Mol)",0.860456
"(Mol, MorganDensity)",0.860228
"(bonds_number, Mol)",0.860171
"(atom_number, Mol)",0.860114
"(doubleBond, Mol)",0.859829


## GRADIENT BOOSTING

In [77]:
df4 = pd.DataFrame(columns = ['Accurancy_GB'], index=to_compare)

In [78]:
for i in df4.index:
    
    new_train = X_train[list(i)]
    new_test = X_test[list(i)]
    
    forest = RandomForestClassifier(n_estimators = 100)
    forest.fit(new_train, y_train.ravel())
    y_pred = forest.predict(new_test)
    df4['Accurancy_GB'].loc[i] = accuracy_score(y_test, y_pred)

In [105]:
tot = df4.rank()['Accurancy_GB'] + df3.rank()['Accurancy_RF'] + df2.rank()['Accurancy']
        

In [107]:
tot = pd.DataFrame(tot)

In [117]:
tot = tot.reset_index()

In [122]:
top_features= tot.explode('index').sort_values(by=[0],ascending=False).groupby('index').sum()

In [127]:
try_features= top_features.sort_values(by=[0],ascending=False).head(6).index

In [128]:
try_features

Index(['Mol', 'LogP', 'MorganDensity', 'bonds_number', 'atom_number',
       'species'],
      dtype='object', name='index')

In [129]:
    new_train = X_train[list(try_features)]
    new_test = X_test[list(try_features)]
    
    neigh = KNeighborsClassifier()
    neigh.fit(new_train, y_train.ravel())
    y_pred = neigh.predict(new_test)
    accuracy_score(y_test, y_pred)

0.8605698005698006

In [131]:
    forest = RandomForestClassifier(n_estimators = 100)
    forest.fit(new_train, y_train.ravel())
    y_pred = forest.predict(new_test)
    accuracy_score(y_test, y_pred)

0.9067806267806268

In [132]:
    forest = RandomForestClassifier(n_estimators = 100)
    forest.fit(new_train, y_train.ravel())
    y_pred = forest.predict(new_test)
    accuracy_score(y_test, y_pred)

0.9065527065527066

In [133]:
try_features = top_features.sort_values(by=[0],ascending=False).index

In [134]:
try_features

Index(['Mol', 'LogP', 'MorganDensity', 'bonds_number', 'atom_number',
       'species', 'ring_numer', 'doubleBond', 'alone_atom_number',
       'conc1_type', 'exposure_type', 'obs_duration_mean', 'tripleBond'],
      dtype='object', name='index')

In [136]:
top_features.sort_values(by=[0],ascending=False)

,0
index,
Mol,2571.0
LogP,2269.0
MorganDensity,1878.5
bonds_number,1654.5
atom_number,1441.0
species,1356.0
ring_numer,1301.0
doubleBond,1074.0
alone_atom_number,1050.0


In [138]:
df1 = pd.DataFrame(columns = try_features, index=['KNN','Random Forest','Gradient Boosting'])
for i in range(1,len(try_features)):
    col = list(try_features[:i])
    l=[]
    new_train = X_train[col]
    new_test = X_test[col]
    
    neigh = KNeighborsClassifier()
    neigh.fit(new_train, y_train.ravel())
    y_pred = neigh.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    forest = RandomForestClassifier(n_estimators = 100)
    forest.fit(new_train, y_train.ravel())
    y_pred = forest.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    grboost = GradientBoostingClassifier()
    grboost.fit(new_train, y_train.ravel())
    y_pred = grboost.predict(new_test)
    l.append(accuracy_score(y_test, y_pred))
    
    df1[X.columns[i]] = l 

In [139]:
df1

index,Mol,LogP,MorganDensity,bonds_number,atom_number,species,ring_numer,doubleBond,alone_atom_number,conc1_type,exposure_type,obs_duration_mean,tripleBond
KNN,0.862165,0.850427,0.856467,0.862336,0.838746,0.843989,0.866610,0.858462,0.843476,0.832080,NaN,0.813504,0.860570
Random Forest,0.907236,0.912308,0.913903,0.905527,0.862393,0.861937,0.905983,0.906382,0.862450,0.861425,NaN,0.859886,0.907692
Gradient Boosting,0.828718,0.829972,0.830484,0.828148,0.817493,0.814758,0.829117,0.825641,0.822165,0.805071,NaN,0.785242,0.826895


In [153]:
X

,exposure_type,obs_duration_mean,conc1_type,species,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,9,0.024211,5,40,0.273831,3,0,0,0.000000,0.0,0.100920,0.666667,-5.9945
1,9,-0.740360,5,208,0.273831,3,0,0,0.000000,0.0,0.100920,0.666667,-5.9945
2,3,0.800083,5,312,0.273831,3,0,0,0.000000,0.0,0.100920,0.666667,-5.9945
3,9,0.800083,5,474,0.273831,3,0,0,0.000000,0.0,0.100920,0.666667,-5.9945
4,3,3.037407,5,312,0.273831,3,0,0,0.000000,0.0,0.100920,0.666667,-5.9945
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53647,3,0.800083,1,372,0.384370,0,0,0,0.522111,1.0,0.089879,1.166667,2.3901
53648,1,-0.571732,3,422,0.473649,0,0,2,0.576717,1.0,0.122888,1.437500,1.3211
53649,9,0.800083,1,262,0.490837,2,0,2,0.609799,2.0,0.162214,1.368421,0.8126
53650,3,0.800083,1,372,0.384370,0,0,2,0.522111,1.0,0.091541,1.083333,0.4780


## FROM HERE WE USE THE DISTANCE MATRIX

In [35]:
categorical = [
"exposure_type", 
"conc1_type","species",
'tripleBond']

In [36]:
non_categorical =[
 'obs_duration_mean',
 'atom_number',
 'alone_atom_number',
 'doubleBond',
 'bonds_number',
 'ring_numer',
 'Mol',
 'MorganDensity',
 'LogP']

In [37]:
X_try  =  X.head(30000)
y_try = y[:30000]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_try, y_try, test_size=0.33, random_state=42)
X_try = X_train.append(X_test)

In [39]:
from new_distance_fast import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
len_X_train = len(X_train)
X_train_new, X_test_new = new_distance_matrix(X_try, len_X_train, categorical,non_categorical, alpha = 5)

In [41]:
X_train_new.shape

(20100, 20100)

In [42]:
X_test_new.shape

(9900, 20100)

In [43]:
neigh = KNeighborsClassifier(metric = 'precomputed')
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)

In [44]:
accuracy_score(y_test, y_pred)

0.888989898989899

In [56]:
a = np.matrix([[0],[1],[2],[3],[2]])
squareform(pdist(a, metric = "euclidean"))

array([[0., 1., 2., 3., 2.],
       [1., 0., 1., 2., 1.],
       [2., 1., 0., 1., 0.],
       [3., 2., 1., 0., 1.],
       [2., 1., 0., 1., 0.]])